In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

from fairlens.bias import utils
from fairlens.bias import metrics
from fairlens.bias.viz import plt_group_dist

In [ ]:
df = pd.read_csv("datasets/compas.csv")

In [ ]:
g1 = {'Ethnicity': ['Caucasian']}
g2 = {'Ethnicity': ['African-American']}
target_attr = 'RawScore'

plt_group_dist(df, target_attr, g1, g2, legend=True)
plt.show()